# Terminal

JupyterLab (and many other applications) use [xtermjs](https://github.com/xtermjs/xterm.js) for rendering line-based terminal outputs. 

In [ ]:
import IPython, tornado.ioloop, json
from wxyz.lab.widget_term import Terminal, W, T
from wxyz.lab.widget_dock import DockBox
from wxyz.html.widget_color import AlphaColorPicker

In [ ]:
def make_a_terminal_demo():
    t0, t1 = t = [Terminal(description=f"Terminal {i}", fit=True) for i in range(2)]
    
    i = 0

    t0.on_data(lambda t0, data: t1.send({"content": "data: " + data["content"] + "\r\n"}))

    def t0_change(change):
        c = i % 5 + 30
        t1.send_line(f"\x1B[1;3;{c}m{change.name}\x1B[0m {change.new}")
    
    t0.observe(t0_change, ("scroll", "selection", "rows", "cols"))

    fit = W.Checkbox(description="fit")
    sliders = {it: W.IntSlider(description=it) for it in ["scroll", "rows", "cols"]}
    
    [T.link((t0, k), (v, "value")) for k, v in sliders.items()]
    T.link((t0, "fit"), (fit, "value"))
    
    colors = {k: AlphaColorPicker(value=v, description=k) for k, v in t0.theme.items()}
    options =  W.VBox([*colors.values(), fit, *sliders.values()])
    options.add_traits(description=T.Unicode("Options").tag(sync=True))
    
    def themed(*kwargs):
        new_theme = {k: w.value for k, w in colors.items()}
        t0.theme = new_theme

    [c.observe(themed, "value") for c in colors.values()];

    box = DockBox([*t, options], layout=dict(height="400px"))

    def hi():
        nonlocal i
        i += 1
        c = i % 5 + 30
        t[0].send_line(f"👋  {i} Hello from \x1B[1;3;{c}mxterm.js\x1B[0m")
    hi()

    pcb = tornado.ioloop.PeriodicCallback(hi, 2000)

    pcb.start()
    
    return box

In [ ]:
if __name__ == "__main__":
    box = make_a_terminal_demo()
    display(box)